In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import glob
import sys
import random
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import datetime
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
import tensorflow_hub as hub
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.manifold import TSNE
plt.rc('figure', figsize = (20, 8))
plt.rc('font', size = 14)
plt.rc('axes.spines', top = False, right = False)
plt.rc('axes', grid = False)
plt.rc('axes', facecolor = 'white')

In [ ]:
#Running tensorflow 2.1.0
tf.__version__

In [ ]:
params = {'sample_perc':1,
          'bs': 128,
          'ps': 0.5,
          'emb_drop': 0.0,
          'epochs': 9, 
          'n_folds': 5,
          'optimizer': 'adam',
          'lr': 1e-03,
          'wd': 0.0001,
          #'trainable': False,
          'hidden_layers': [128],
          'save_model': False
         }

In [ ]:
dataset_pth = 'DATA_PTH'
results_pth = 'experiment_results/'
if not os.path.exists(results_pth):
    os.makedirs(results_pth)

In [ ]:
target = 'DEPENDENT_VARIABLE'

In [ ]:
def random_seed(seed_value, use_cuda=True):
    np.random.seed(seed_value) # cpu vars
    random.seed(seed_value) # Python
    tf.random.set_seed(seed_value) #tensorflow

In [ ]:
random_seed(123)

# Prepare dataset

In [ ]:
def preprocess_data(sample_perc=1): 
    '''Load and preprocess data'''
    
    df = pd.read_csv(dataset_pth + 'DATASET.csv', sep=',')
    #remove high cardinal features with over 500 unique values
    for col in df.columns:
        if (df[col].nunique()>500) & (col!='TEXT_VARIABLE'):
            print(col, df[col].nunique())
            df.drop(col, axis=1, inplace=True)
            
    if sample_perc < 1:
        #use sample of the data
        np.random.seed(123)
        sampler = StratifiedShuffleSplit(train_size=sample_perc, n_splits=1)
        sample,_ = next(sampler.split(df.loc[:,df.columns!=target], df[target]))
        df = df.loc[sample].reset_index(drop=True)
        print(f'Using {sample_perc*100}% of the data')
        
        random_seed(123)
        stratified_split = StratifiedKFold(5, shuffle=True, random_state=123)
        train_indexes, test_indexes = next(stratified_split.split(df.loc[:,df.columns!=target], df[target]))
    else:
        #load split indexes
        from numpy import genfromtxt
        train_indexes = genfromtxt(dataset_pth + 'TRAINING_IDXS.csv', delimiter=',', dtype=float).astype(int)
        test_indexes = genfromtxt(dataset_pth + 'TESTING_IDXS.csv', delimiter=',', dtype=float).astype(int)
    
    #split data into train and test set
    train = df.loc[train_indexes]
    test = df.loc[test_indexes]
    return (df, train, test)

In [ ]:
df, train, test = preprocess_data(params['sample_perc'])
df.shape, train.shape, test.shape

In [ ]:
#assign variables to data types
cat_vars = ['CAT_VARS']

cont_vars = ['CONT_VARS']

text_var = 'TEXT_VAR'

all_features = cat_vars + cont_vars + [text_var]
print(len(all_features))

In [ ]:
# Scaling the numerical features
scaler = preprocessing.MinMaxScaler().fit(train[cont_vars])
train[cont_vars] = scaler.transform(train[cont_vars])
test[cont_vars] = scaler.transform(test[cont_vars])

#Categorify categorical features
for col in cat_vars+[text_var, target]:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)
    df[col] = df[col].astype(str)
    
#lowercase the text feature
train[text_var] = train[text_var].apply(lambda x: x.lower())
test[text_var] = test[text_var].apply(lambda x: x.lower())
    
CLASS_LABELS =  np.array(df[target].unique())

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle:bool=True, batch_size=32):
    '''Create tf.data dataset from pandas DataFrame
    
    Parameters:
    dataframes: pandas DataFrame
    shuffle: Boolean indicating whether to shuffle the data
    batch_size: batch size
    
    Returns a tf.data dataset
    '''
    
    dataframe = dataframe.copy()
    labels = dataframe.pop(target)
    #create one-hot encoded label vector
    labels = labels.apply(lambda x:x == CLASS_LABELS)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    if batch_size == None:
        # complete data in 1 batch
        ds = ds.batch(len(labels))
    else:
        ds = ds.batch(batch_size)
    del dataframe
    return ds

# Neural Network

## Embedding dimensions

In [ ]:
#dimensions calculated by fastai rule
dimension_limit = 40 #initial: 600
print('Embedding sizes based on fastai rule')
emb_szs = {}
for column in df[cat_vars]:
    n_cat = df[column].nunique()
    emb_szs[column] = min(dimension_limit, round(1.6 * n_cat**0.56))
pprint(emb_szs)
params['emb_szs'] = emb_szs

In [ ]:
#set dimension of highest cardinal feature to specific value
'''
emb_szs['C3_ENHANCED_MATERIAL_GROUP'] = 439
params['emb_szs'] = emb_szs
pprint(emb_szs)
'''

## Feature Columns

In [ ]:
#Create feature columns and the final input for the neural network

# Numeric Columns
numeric_columns = {
    col : feature_column.numeric_column(col) \
          for col in cont_vars
}

# Categorical Columns
categorical_columns = {
    col : feature_column.categorical_column_with_vocabulary_list(col, df[col].unique().tolist()) \
          for col in cat_vars
}


#embeddings: uncomment this section (and comment section below) to use entity embeddings
params['encoding'] = 'Entity Embeddings'
for col in categorical_columns:
    categorical_columns[col] = feature_column.embedding_column(categorical_columns[col], dimension=emb_szs[col])

    
#one-hot encoded: uncomment this section (and comment section above) to use one-hot encoding
'''
# One-Hot Encoding
params['encoding'] = 'One-Hot Encoding'
for col in categorical_columns:
    categorical_columns[col] = feature_column.indicator_column(categorical_columns[col])
'''


#CATEGORICAL AND NUMERIC INPUTS - uncomment this section to use both (comment sections below)
'''
#capture all feature columns in one vector
feature_columns = list(numeric_columns.values()) + list(categorical_columns.values())

#prepare final inputs
input_tabular = {
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32') \
          for colname in numeric_columns.keys()
}
input_tabular.update({
    colname : tf.keras.layers.Input(name=colname, shape=(),  dtype='string') \
          for colname in categorical_columns.keys()
})
'''

#ONLY NUMERIC INPUTS - uncomment this section (comment sections above and below)
'''
feature_columns = list(numeric_columns.values())
input_tabular = {
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32') \
          for colname in numeric_columns.keys()
}
'''

#ONLY CATEGORICAL INPUTS - uncomment this section (comment sections above)
feature_columns = list(categorical_columns.values())
input_tabular = {
    colname : tf.keras.layers.Input(name=colname, shape=(),  dtype='string') \
          for colname in categorical_columns.keys()
}



#NLP input:
input_nlp = tf.keras.layers.Input(name=text_var, shape=(), dtype = tf.string)

In [ ]:
class SkipCon(keras.layers.Layer):
    def __init__(self, size, reduce = True, deep = 3, skip_when=0, activation="relu", kernel_regularizer=0.0, **kwargs):
        """
        Class for skip connections
        
        @Params
        size = size of dense layer
        deep = the depth of network in one SkipCon block call
        skip_when =  if a skip connection is required, pass 1
        activation = by default using relu
        """    
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation) 
        self.main_layers = []
        self.skip_when = skip_when 
        if reduce:
            for _ in range(deep):
                self.main_layers.extend([
                      keras.layers.Dense(size, activation=activation, use_bias=True, kernel_regularizer=tf.keras.regularizers.l2(kernel_regularizer)),
                      keras.layers.BatchNormalization()])

                # Reduce the input size by two each time
                size = size/2
        else:
            for _ in range(deep):
                self.main_layers.extend([
                    keras.layers.Dense(size, activation=activation, use_bias=True, kernel_regularizer=tf.keras.regularizers.l2(kernel_regularizer)),
                    keras.layers.BatchNormalization()])

        self.skip_layers = []
        if skip_when > 0:
            if reduce:
                size = size*2 # since the size of skipped connection should match with cascaded dense
            self.skip_layers = [
                    keras.layers.Dense(size, activation=activation,use_bias=True, kernel_regularizer=tf.keras.regularizers.l2(kernel_regularizer)),
                    keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        if not self.skip_when:
            return self.activation(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
#Prepare evaluation metrics
METRICS = [keras.metrics.TruePositives(name='tp'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.TrueNegatives(name='tn'),
           keras.metrics.FalseNegatives(name='fn'), 
           keras.metrics.BinaryAccuracy(name='binary_accuracy'),
           keras.metrics.CategoricalAccuracy(name='cat_accuracy'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
    ]

def get_optimizer():
    '''Function that returns an optimzer based on the parameters for the model'''
    
    if params['optimizer'] == 'sgd':
        optimizer = keras.optimizers.SGD(lr=params['lr'], momentum=params['momentum'], decay=params['wd'])
    elif params['optimizer'] == 'adam':
        optimizer = keras.optimizers.Adam(lr=params['lr'])
    elif params['optimizer'] == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(lr=params['lr'], momentum=params['momentum'])
    else:
        raise Exception('Wrong input for optimizer parameter given.')
    return optimizer

#early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        min_delta=1e-03,
        verbose=1,
        patience=2,
        mode='min',
        restore_best_weights=True)

#tensorboard callback
tensorboard_pth = 'logs//fit//' 
log_dir = tensorboard_pth + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
if not os.path.exists(tensorboard_pth):
    os.makedirs(tensorboard_pth)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch', profile_batch=0)

In [ ]:
def create_tabular_model():
    '''Function that creates and compiles the standard neural network (categorical, numeric or both inputs)'''
    
    # Create a feature layer = input layer
    feature_layer = keras.layers.DenseFeatures(feature_columns)(input_tabular)
    #hidden layers
    tabular_layer = keras.layers.Dense(params['hidden_layers'][0], activation='relu', use_bias = True, kernel_regularizer=tf.keras.regularizers.l2(params['wd']))(feature_layer)
    tabular_layer = keras.layers.Dropout(params['ps'])(tabular_layer)
    if len(params['hidden_layers'])>1:
        tabular_layer = keras.layers.Dense(params['hidden_layers'][1], activation='relu', use_bias = True, kernel_regularizer=tf.keras.regularizers.l2(params['wd']))(tabular_layer)
        tabular_layer = keras.layers.Dropout(params['ps'])(tabular_layer)
    if len(params['hidden_layers'])>2:
        tabular_layer = keras.layers.Dense(params['hidden_layers'][2], activation='relu', use_bias = True, kernel_regularizer=tf.keras.regularizers.l2(params['wd']))(tabular_layer)
        tabular_layer = keras.layers.Dropout(params['ps'])(tabular_layer)

    #output layer
    z = keras.layers.Dense(len(CLASS_LABELS), activation="softmax")(tabular_layer)

    random_seed(123)
    model = keras.Model(inputs=[input_tabular], outputs=z)

    optimizer = get_optimizer()
    
    #compile network
    random_seed(123)
    model.compile(optimizer= optimizer,
                  loss='categorical_crossentropy',
                  metrics=METRICS)
    return model

In [ ]:
def create_tabular_skip_model():
    '''Function that creates and compiles the standard neural network with skip connections (categorical, numeric or both inputs)'''
    
    # Create a feature layer = input layer
    feature_layer = keras.layers.DenseFeatures(feature_columns)(input_tabular)
    #hidden layers
    tabular_layer = keras.layers.Dense(params['hidden_layers'][0], activation='relu', use_bias = True, kernel_regularizer=tf.keras.regularizers.l2(params['wd']))(feature_layer)
    tabular_layer = keras.layers.Dropout(params['ps'])(tabular_layer)
    if len(params['hidden_layers'])>1:
        tabular_layer = SkipCon(size = params['hidden_layers'][1], deep = 1, reduce = True, skip_when=1, activation="relu", kernel_regularizer=params['wd'])(tabular_layer)
        tabular_layer = keras.layers.Dropout(params['ps'])(tabular_layer)
    if len(params['hidden_layers'])>2:
        tabular_layer = SkipCon(size = params['hidden_layers'][2], deep = 1, reduce = False, skip_when=1, activation="relu", kernel_regularizer=params['wd'])(tabular_layer)
        tabular_layer = keras.layers.Dropout(params['ps'])(tabular_layer)
    #output layer
    z = keras.layers.Dense(len(CLASS_LABELS), activation="softmax")(tabular_layer)

    random_seed(123)
    model = keras.Model(inputs=[input_tabular], outputs=z)

    optimizer = get_optimizer()
    
    random_seed(123)
    model.compile(optimizer= optimizer,
                  loss='categorical_crossentropy',
                  metrics=METRICS)
    return model

In [ ]:
def create_nlp_model():
    '''Function that creates and compiles the NLP network (text input)'''
    
    #load pre-trained model (USE)
    print('loading language model')
    embedding = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=params['trainable'] , dtype=tf.string, input_shape=[], output_shape=[512])(input_nlp)
    nlp_layer = keras.layers.Dense(256, activation='relu')(embedding)
    nlp_layer = keras.layers.Dense(64, activation='relu')(nlp_layer)
    nlp_layer = keras.layers.Dense(16, activation='relu')(nlp_layer)
    
    #output layer
    z = keras.layers.Dense(len(CLASS_LABELS), activation="softmax")(nlp_layer)

    random_seed(123)
    model = keras.Model(inputs=[input_nlp], outputs=z)

    optimizer = get_optimizer()
    
    random_seed(123)
    model.compile(optimizer= optimizer,
                  loss='categorical_crossentropy',
                  metrics=METRICS)
    return model

In [ ]:
def create_tabular_nlp_model():
    '''Function that creates and compiles the combined neural network (categorical, (numeric) and text inputs)'''
    
    # Create a feature layer = input layer for categorical variables (and numeric)
    feature_layer = keras.layers.DenseFeatures(feature_columns)(input_tabular)
    #hidden layers 
    tabular_layer = keras.layers.Dense(128, activation='relu', use_bias = True, kernel_regularizer=tf.keras.regularizers.l2(params['wd']))(feature_layer)
    tabular_layer = keras.layers.Dropout(params['ps'])(tabular_layer)

    #load pre-trained NLP model (USE)
    print('loading language model')
    embedding = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=params['trainable'] , dtype=tf.string, input_shape=[], output_shape=[512])
    embedding = embedding(input_nlp)
    #hidden layers
    nlp_layer = keras.layers.Dense(256, activation='relu')(embedding)
    nlp_layer = keras.layers.Dense(128, activation='relu')(nlp_layer)

    # combine the output of the two branches
    combined = tf.concat([tabular_layer, nlp_layer], axis =-1)

    # addition feed-forward layers
    both_layer = keras.layers.Dense(64, activation='relu', use_bias = True, kernel_regularizer=tf.keras.regularizers.l2(params['wd']))(combined)
    both_layer = keras.layers.Dropout(0.5)(both_layer)
    both_layer = keras.layers.Dense(32, activation='relu', use_bias = True, kernel_regularizer=tf.keras.regularizers.l2(params['wd']))(both_layer)
    
    #output layer
    z = keras.layers.Dense(len(CLASS_LABELS), activation="softmax")(both_layer)

    random_seed(123)
    model = keras.Model(inputs=[input_tabular, input_nlp], outputs=z)

    optimizer = get_optimizer()
    
    random_seed(123)
    model.compile(optimizer= optimizer,
                  loss='categorical_crossentropy',
                  metrics=METRICS)
    return model

**Helper functions:**

In [ ]:
def order_history(history):
    train_dict=dict()
    val_dict=dict()
    for (key, value) in history.items():
       # Check if key is even then add pair to new dictionary
        if key.split('_')[0] == 'val':
            val_dict[key] = value
        else:
            train_dict[key] = value
    ordered_history = train_dict.copy()
    ordered_history.update(val_dict)
    return ordered_history

def add_fold_to_dict(history, dt):
    history = history.copy()
    if dt == {}:
        dt = dict(history)
    else:
        for key in dt.keys():
            dt[key].extend(history[key])
    return dt

def get_avg_column_val(df):
    averages = list()
    for i in range(len(df.columns)):
        averages.append(df[i].mean())
    return averages

def kfold_results(dt, n_folds, epochs):
    averages = dict()
    for key in dt.keys():
        df = pd.DataFrame(pd.Series(dt[key]).values.reshape(n_folds,epochs))
        averages[key] = get_avg_column_val(df)
    epochs = np.arange(epochs)
    results_lists = {'epochs': epochs}
    results_lists.update(averages)
    results = pd.DataFrame(results_lists)
    return results
    
def plot_kfold_results(results):
    nb_epochs=results.shape[0]
    fig,ax = plt.subplots(2,1,figsize=(8,12))
    fig.suptitle('Results - averaged over folds')
    ax[0].plot(list(range(nb_epochs)), results['loss'], label='Training loss')
    ax[0].plot(list(range(nb_epochs)), results['val_loss'], label='Validation loss')
    ax[0].set_xlabel('Epoch')
    ax[0].xaxis.set_ticks(np.arange(0,nb_epochs,1))
    ax[0].set_ylabel('Loss')
    ax[0].legend(loc='best')
    ax[1].plot(list(range(nb_epochs)),results['cat_accuracy'], label='Training Accuracy')
    ax[1].plot(list(range(nb_epochs)),results['val_cat_accuracy'], label='Validation Accuracy')
    ax[1].xaxis.set_ticks(np.arange(0,nb_epochs,1))
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Accuracy / %')
    ax[1].legend(loc='best')

def print_results(cv_results, test_results=None):
    print('\n')
    print('-'*15)
    print('Test Set Results:')
    print('\n')
    pprint(test_results)
    
    print('\n')
    print('-'*15)
    print('Cross-Validation Results (averaged over folds):')
    print('\n')
    print(cv_results)
    plot_kfold_results(cv_results)

In [ ]:
def get_labels(labels):
    '''Return one-hot encoded labels'''
    labels = labels.apply(lambda x:x == CLASS_LABELS)
    labels *= 1
    return labels

def calc_metrics(logs:dict, predict_probs, target, validation=False, test=False):
    '''
    Function to calculate the evaluation metrics
    
    Parameters:
    logs: dict with logs
    predict: prediction probabilities
    target: target labels
    validation: True if needed to calculate validation metrics
    '''
    
    #get actual class prediction and target
    predict_classes = np.argmax(predict_probs, axis=1)
    target_classes = np.argmax(target, axis=1)
    
    if validation:
        prefix = 'val_'
    elif test:
        prefix = 'test_'
        #create confusion matrix
        conf_targets = [CLASS_LABELS[target] for target in target_classes]
        conf_predict = [CLASS_LABELS[predict] for predict in predict_classes]            
        conf_matrix = confusion_matrix(conf_targets, conf_predict, labels=CLASS_LABELS)
        heat_plot = sns.heatmap(conf_matrix, annot=True, cmap='coolwarm', xticklabels=CLASS_LABELS.astype(float).astype(int).astype(str), yticklabels=CLASS_LABELS.astype(float).astype(int).astype(str))
        figure = heat_plot.get_figure()
        figure.savefig(results_pth  + 'confusion_matrix.png')
    else:
        prefix = '' 
    metrics = dict()
    metrics[prefix + 'f1_micro'] = f1_score(target_classes, predict_classes, average='micro')
    metrics[prefix + 'f1_macro'] = f1_score(target_classes, predict_classes, average='macro')
    metrics[prefix + 'auc_micro'] = roc_auc_score(target, predict_probs, average='micro')
    metrics[prefix + 'auc_macro'] = roc_auc_score(target, predict_probs, average='macro', multi_class='ovr')

    #prepare print message
    message = ''
    message = [message + f' - {metric}: {metrics[metric]}' for metric in metrics.keys()]

    #add calculated metrics to logs
    for metric in metrics.keys():
        logs[metric] = metrics[metric]

    return logs, message


class Metrics(Callback):
    '''Callback to calculate additional metrics'''
    
    def __init__(self, training_data, train_targ, validation_data, val_targ):
        super(Callback, self).__init__()
        self.training_data = training_data
        self.train_targ = train_targ
        self.validation_data = validation_data
        self.val_targ = val_targ
        
    def on_epoch_end(self, epoch, logs):
        #predict on train set
        train_predict_probs = np.asarray(self.model.predict(self.training_data))
        #targets
        train_target = list(self.train_targ)
        #calculate metrics
        logs, message_train = calc_metrics(logs, predict_probs=train_predict_probs, target=train_target)
        
        #clear memory
        for i in range(10):
            gc.collect()
            
        #metrics for validation set:
        val_predict_probs = np.asarray(self.model.predict(self.validation_data))
        val_target = list(self.val_targ)
        logs, message_val = calc_metrics(logs, predict_probs=val_predict_probs, target=val_target, validation=True)
        
        print(message_train + message_val)
        return

class Test_Metrics(Callback):
    '''Callback to calculate metrics for test set'''
    
    def __init__(self, test_data, test_targ):
        super(Callback, self).__init__()
        self.test_data = test_data
        self.test_targ = test_targ
        self.metrics = dict()
        
    def on_test_end(self, logs):
        #calculate metrics for test set
        test_predict_probs = np.asarray(self.model.predict(self.test_data))
        test_target = list(self.test_targ)
        self.metrics, message = calc_metrics(self.metrics, predict_probs=test_predict_probs, target=test_target, test=True)
        return

In [ ]:
def fit_model_test(train, test, model_fn, epochs, bs): 
    '''
    Function to fit model on complete training data and evaluate on test data
    
    Parameters:
    train: training data as pandas DataFrame
    test: testing data as pandas DataFrame
    model_fn: function name that creates the neural network
    epochs: number of epochs to train
    bs: batch size
    
    Returns evaluation results and training history
    '''
    
    #get TF datasets from pandas DataFrame
    random_seed(123)
    train_ds = df_to_dataset(train, shuffle=False, batch_size=bs)
    test_ds = df_to_dataset(test, shuffle=False, batch_size=None)
    
    #create model
    random_seed(123)
    model = model_fn()
    
    #fit model on complete training data
    random_seed(123)
    history = model.fit(train_ds, validation_data=test_ds, epochs=epochs, callbacks=[early_stopping])
    #history = model.fit(train_ds, validation_data=test_ds, epochs=epochs, callbacks=[early_stopping, tensorboard_callback])
    
    #evaluate the network with the test set:
    test_results = dict()
    #get one-hot encoded targets
    test_targ = get_labels(test[target])
    #prepare callback
    test_metrics = Test_Metrics(test_ds, test_targ)
    #evaluate
    evaluation = model.evaluate(test_ds, callbacks=[test_metrics])
    #add metrics to test results
    for idx, metric in enumerate(model.metrics_names):
        test_results['test_' + metric] = evaluation[idx]
    test_results.update(test_metrics.metrics)
    print(test_results)
    
    #save model
    if params['save_model']:
        model.save(results_pth + 'model')
    
    #clear session
    keras.backend.clear_session()
    return (test_results, history)

In [ ]:
def fit_model_cv(train, model_fn, epochs, n_folds, bs, test=None, elongation=None):
    '''
    Function to fit model on training data with cross-validation
    
    Parameters:
    train: training data as pandas DataFrame
    model_fn: function name that creates the neural network
    epochs: number of epochs to train
    n_folds: number of folds into which the training data should be splitted
    bs: batch size
    test: optional - if given, additionally, a neural network is trained on the complete training data and evaluated on the test data
    '''
        
    #tracking variable
    folds = dict()
    
    #prepare cross validation
    random_seed(123)
    stratified_k_fold = StratifiedKFold(n_folds, shuffle=True, random_state=1)
    
    #iterate over folds
    for iteration_idx, (train_idxs, valid_idxs) in enumerate(stratified_k_fold.split(train.loc[:,train.columns!=target], train[target])):
        fold_idx = iteration_idx+1
        print('-'*20, '\n', f'> Fold: {fold_idx}'); print('-'*20)
        
        #get training and validation sets
        train_df = train.iloc[train_idxs].copy()
        val_df = train.iloc[valid_idxs].copy()
        #transform DataFrames into TF datasets
        train_ds = df_to_dataset(train_df, shuffle=False, batch_size=bs)
        val_ds = df_to_dataset(val_df, shuffle=False, batch_size=bs*2)
        
        #add additional metrics  (callback)
        train_labels = train_df[target]
        val_labels = val_df[target]
        metrics = Metrics(training_data=train_ds, train_targ=get_labels(train_labels), validation_data=val_ds, val_targ=get_labels(val_labels))
    
        #get model
        random_seed(123)
        model = model_fn()
        if fold_idx == 1: print(model.summary())
        
        #model fitting
        random_seed(123)
        history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[metrics])
        #history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[metrics, tensorboard_callback])
        history.history = order_history(history.history)
        
        #add fold to dict
        folds = add_fold_to_dict(history.history, folds)
        
        #free-up memory
        keras.backend.clear_session()
        for i in range(30):
            gc.collect()
        del model
        del history
        del metrics
        del train_df
        del train_ds
        del val_df
        del val_ds
    
    #get results    
    results = kfold_results(folds, n_folds, epochs)
    experiment_results = [results]
    
    if test is not None:
         #train a network on complete training set and evaluate on test set
        print('-'*15)
        print('Test Set: \n')
        
        #create, fit and evaluate network
        test_results, history_test = fit_model_test(train, test, params['model_fn'], epochs=epochs, bs=params['bs'])
        
        #get trained epochs in case network stopped early through callback
        final_epochs = epochs if early_stopping.stopped_epoch==0 else (early_stopping.stopped_epoch-early_stopping.patience+1)
        print('Early Stopping:')
        print(early_stopping.stopped_epoch)
        
        experiment_results.append(test_results)
        
    #show results
    if test is not None:
        print_results(experiment_results[0], experiment_results[1])
    else:
        print_results(experiment_results[0])

In [ ]:
params['model_fn'] = create_tabular_model
params['epochs'] = 9
fit_model_cv(train, test=test, model_fn=params['model_fn'], epochs=params['epochs'], n_folds=params['n_folds'], bs=params['bs'])